In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
pmids = pd.read_csv('pmids.txt', error_bad_lines=False, sep='    ')

Skipping line 33306: Expected 2 fields in line 33306, saw 4. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 33307: Expected 2 fields in line 33307, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 33597: Expected 2 fields in line 33597, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 40067: Expected 2 fields in line 40067, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 40068: Expected 2 fields in line 40068, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 40069: Expected 2 fields in line 40069, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 40070: Expected 2 fields in line 40070, saw 3. Error could possibly be due to quotes b

In [3]:
pmids.head()

,./NCT0000xxxx/NCT00000113.xml:,<PMID>12657584</PMID>
0,./NCT0000xxxx/NCT00000113.xml:,<PMID>15223788</PMID>
1,./NCT0000xxxx/NCT00000113.xml:,<PMID>16009841</PMID>
2,./NCT0000xxxx/NCT00000114.xml:,<PMID>8512476</PMID>
3,./NCT0000xxxx/NCT00000114.xml:,<PMID>8675413</PMID>
4,./NCT0000xxxx/NCT00000115.xml:,<PMID>8684794</PMID>


In [4]:
pmids.shape

(314821, 2)

In [5]:
pmids.columns =['NCTID','PMIDs']

In [6]:
pmids = pmids.dropna(how = 'any')

In [7]:
pmids.shape

(314783, 2)

In [8]:
pmids = pmids[pmids.PMIDs.str.contains("(\d+)") == True]

In [9]:
pmids['PMID'] = pmids.iloc[:,1].str.extract('(\d+)').astype(str)

In [10]:
pmids.shape

(314779, 3)

In [11]:
pmids.head()

,NCTID,PMIDs,PMID
0,./NCT0000xxxx/NCT00000113.xml:,<PMID>15223788</PMID>,15223788
1,./NCT0000xxxx/NCT00000113.xml:,<PMID>16009841</PMID>,16009841
2,./NCT0000xxxx/NCT00000114.xml:,<PMID>8512476</PMID>,8512476
3,./NCT0000xxxx/NCT00000114.xml:,<PMID>8675413</PMID>,8675413
4,./NCT0000xxxx/NCT00000115.xml:,<PMID>8684794</PMID>,8684794


In [12]:
pmcids = pd.read_csv('PMC-ids.csv', error_bad_lines=False, usecols=['PMCID', 'PMID'],dtype=str)

In [13]:
pmcids.head()

,PMCID,PMID
0,PMC13900,11250746
1,PMC13901,11250747
2,PMC13902,11250748
3,PMC13911,11056684
4,PMC13912,11400682


In [14]:
mappingTable = pd.merge(pmcids, pmids, left_on='PMID', right_on='PMID')

In [15]:
mappingTable.head()

,PMCID,PMID,NCTID,PMIDs
0,PMC14630,11136232,./NCT0002xxxx/NCT00025831.xml:,<PMID>11136232</PMID>
1,PMC14638,11149951,./NCT0064xxxx/NCT00645580.xml:,<PMID>11149951</PMID>
2,PMC14647,11209064,./NCT0174xxxx/NCT01741246.xml:,<PMID>11209064</PMID>
3,PMC14647,11209064,./NCT0208xxxx/NCT02083367.xml:,<PMID>11209064</PMID>
4,PMC14647,11209064,./NCT0272xxxx/NCT02729428.xml:,<PMID>11209064</PMID>


In [16]:
mappingTable.shape

(66689, 4)

In [17]:
mappingTable.to_csv("mappingTable.csv", index=False)

In [18]:
alloc = pd.read_csv('alloc.txt', error_bad_lines=False, sep='    ')

Skipping line 4350: Expected 2 fields in line 4350, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4351: Expected 2 fields in line 4351, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4352: Expected 2 fields in line 4352, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4353: Expected 2 fields in line 4353, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4354: Expected 2 fields in line 4354, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4355: Expected 2 fields in line 4355, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 4356: Expected 2 fields in line 4356, saw 3. Error could possibly be due to quotes being ignored w

In [19]:
alloc.columns = ['NCTID','Randomized']

In [20]:
alloc.head()

,NCTID,Randomized
0,./NCT0000xxxx/NCT00000108.xml:,<allocation>Randomized</allocation>
1,./NCT0000xxxx/NCT00000111.xml:,<allocation>Randomized</allocation>
2,./NCT0000xxxx/NCT00000113.xml:,<allocation>Randomized</allocation>
3,./NCT0000xxxx/NCT00000114.xml:,<allocation>Randomized</allocation>
4,./NCT0000xxxx/NCT00000115.xml:,<allocation>Randomized</allocation>


In [21]:
alloc['RCT'] = alloc.Randomized == '<allocation>Randomized</allocation>'

In [22]:
alloc.head()

,NCTID,Randomized,RCT
0,./NCT0000xxxx/NCT00000108.xml:,<allocation>Randomized</allocation>,True
1,./NCT0000xxxx/NCT00000111.xml:,<allocation>Randomized</allocation>,True
2,./NCT0000xxxx/NCT00000113.xml:,<allocation>Randomized</allocation>,True
3,./NCT0000xxxx/NCT00000114.xml:,<allocation>Randomized</allocation>,True
4,./NCT0000xxxx/NCT00000115.xml:,<allocation>Randomized</allocation>,True


In [23]:
sum(alloc.RCT==True)

139893

In [24]:
alloc.shape

(170927, 3)

In [25]:
tmpTable = pd.merge(mappingTable, alloc, left_on='NCTID', right_on='NCTID')

In [26]:
tmpTable.shape

(44913, 6)

In [27]:
tmpTable.head()

,PMCID,PMID,NCTID,PMIDs,Randomized,RCT
0,PMC14708,11158594,./NCT0302xxxx/NCT03022253.xml:,<PMID>11158594</PMID>,intervention or control group. The investiga...,False
1,PMC14708,11158594,./NCT0302xxxx/NCT03022253.xml:,<PMID>11158594</PMID>,allocation group.,False
2,PMC14708,11158594,./NCT0302xxxx/NCT03022253.xml:,<PMID>11158594</PMID>,<allocation>Randomized</allocation>,True
3,PMC16938,10944235,./NCT0302xxxx/NCT03022253.xml:,<PMID>10944235</PMID>,intervention or control group. The investiga...,False
4,PMC16938,10944235,./NCT0302xxxx/NCT03022253.xml:,<PMID>10944235</PMID>,allocation group.,False


In [28]:
pd.pivot_table(tmpTable, values='PMID', columns=['RCT'], aggfunc='count')

RCT,False,True
PMID,8694,36219


In [29]:
#!mkdir RCTData
src = 'PubMed'
dest = 'RCTData/'

In [30]:
pubFiles = pd.DataFrame()
pubFiles['PMCFILE'] = os.listdir(src)

In [31]:
tmpTable['PMCFILE'] = tmpTable.PMCID+ '.nxml'

In [32]:
trainingTable = pd.merge(tmpTable, pubFiles, left_on='PMCFILE', right_on='PMCFILE')

In [33]:
trainingTable = trainingTable.drop_duplicates(subset='PMCID')
pd.pivot_table(trainingTable, values='PMID', columns=['RCT'], aggfunc='count')

RCT,False,True
PMID,1627,8589


In [34]:
trainingTable.head()

,PMCID,PMID,NCTID,PMIDs,Randomized,RCT,PMCFILE
0,PMC2193278,10974034,./NCT0050xxxx/NCT00505375.xml:,<PMID>10974034</PMID>,<allocation>Randomized</allocation>,True,PMC2193278.nxml
1,PMC138697,11597323,./NCT0040xxxx/NCT00405678.xml:,<PMID>11597323</PMID>,<allocation>Randomized</allocation>,True,PMC138697.nxml
2,PMC400672,15084243,./NCT0040xxxx/NCT00405678.xml:,<PMID>15084243</PMID>,<allocation>Randomized</allocation>,True,PMC400672.nxml
3,PMC521689,15377394,./NCT0092xxxx/NCT00928135.xml:,<PMID>15377394</PMID>,<allocation>Randomized</allocation>,True,PMC521689.nxml
5,PMC1386668,16483382,./NCT0092xxxx/NCT00928135.xml:,<PMID>16483382</PMID>,<allocation>Randomized</allocation>,True,PMC1386668.nxml


In [35]:
# copy all available training files into folder dest
for fileName in trainingTable['PMCFILE']:
    full_file_name = os.path.join(src, fileName)
    shutil.copy(full_file_name, dest)

In [36]:
trainingTable.to_csv("trainingTable.csv", index=False)

In [37]:
fileList = trainingTable.PMCID

In [38]:
len(fileList)

10216

In [39]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 1, 32, 32).astype(np.float32)

In [40]:
trainingTable = pd.read_csv('trainingTable.csv')

In [41]:
trainingTable.head()

,PMCID,PMID,NCTID,PMIDs,Randomized,RCT,PMCFILE
0,PMC2193278,10974034,./NCT0050xxxx/NCT00505375.xml:,<PMID>10974034</PMID>,<allocation>Randomized</allocation>,True,PMC2193278.nxml
1,PMC138697,11597323,./NCT0040xxxx/NCT00405678.xml:,<PMID>11597323</PMID>,<allocation>Randomized</allocation>,True,PMC138697.nxml
2,PMC400672,15084243,./NCT0040xxxx/NCT00405678.xml:,<PMID>15084243</PMID>,<allocation>Randomized</allocation>,True,PMC400672.nxml
3,PMC521689,15377394,./NCT0092xxxx/NCT00928135.xml:,<PMID>15377394</PMID>,<allocation>Randomized</allocation>,True,PMC521689.nxml
4,PMC1386668,16483382,./NCT0092xxxx/NCT00928135.xml:,<PMID>16483382</PMID>,<allocation>Randomized</allocation>,True,PMC1386668.nxml


In [42]:
data_X = []
data_Y = []

# Load stop-words
stop_words = set(stopwords.words('english'))
stop_words.update(('name','surname','given','first'))

# Initialize tokenizer
regexp_tokenizer = RegexpTokenizer('[\'a-zA-Z]+')

# Initialize lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [43]:
# Tokenize a document to words
def tokenize(document, rebuild_document=True):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [wordnet_lemmatizer.lemmatize(t.lower()) 
                  for t in regexp_tokenizer.tokenize(sentence) if t.lower() not in stop_words]
        words += tokens

    if rebuild_document:
        return ' '.join(words).strip()
    else:
        return words

In [44]:
# put all tokens into dataset data_X and assign the correspond labels
for i in tqdm.tqdm(range(len(trainingTable['PMCID']))):
    fileName = os.path.join(dest, trainingTable['PMCFILE'][i])
    with open(fileName, 'r',encoding='ISO-8859-1') as data_file:
        rawdata = data_file.read()
    soup = BeautifulSoup(rawdata)
    text = soup.get_text()
    data_X.append(tokenize(text))
    data_Y.append(trainingTable['RCT'][i])

100%|██████████| 10216/10216 [25:39<00:00,  6.63it/s]


In [45]:
p_file = 'data_X'

with open(p_file, 'wb') as fout:
    pickle.dump(data_X, fout)

In [46]:
# Create tagged documents
taggedDoc = []

for i, document in enumerate(data_X):
    taggedDoc.append(TaggedDocument(document.split(' '), [i]))

p_file = 'taggedDoc'
with open(p_file, 'wb') as fout:
    pickle.dump(taggedDoc, fout)

# Train Doc2Vec model
# https://arxiv.org/pdf/1405.4053v2.pdf
doc2vecSize = 32*32

doc2vecModel = Doc2Vec(documents=taggedDoc, size=doc2vecSize, window=3, 
                        min_count=2, iter=30, workers=multiprocessing.cpu_count())
doc2vecModel.save('doc2vecModel')
doc2vecModel.init_sims(replace=False)

# Vectorize documents and split corpus to training and testing
vectors = []

for document in data_X:
    vectors.append(doc2vecModel.infer_vector(document.split(' ')))

d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [47]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [48]:
p_file = 'RCT_Vectors'

with open(p_file, 'wb') as fout:
    pickle.dump(vectors, fout)

In [49]:
p_file = 'RCT_labels'

with open(p_file, 'wb') as fout:
    pickle.dump(data_Y, fout)

In [50]:
# creating 2d and 4d training/testing dataset
batch_size = 64
train_iter = mx.io.NDArrayIter(X_train_array, Y_train_array, batch_size=batch_size, label_name='svm_label')
test_iter = mx.io.NDArrayIter(X_test_array, Y_test_array, batch_size=batch_size, label_name='svm_label')

train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

In [63]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.render('DNN + SVM')

'DNN + SVM.pdf'

In [64]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.1,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Train-accuracy=0.841923
INFO:root:Epoch[0] Time cost=0.061
INFO:root:Epoch[0] Validation-accuracy=0.833462
INFO:root:Epoch[1] Train-accuracy=0.843205
INFO:root:Epoch[1] Time cost=0.056
INFO:root:Epoch[1] Validation-accuracy=0.833462
INFO:root:Epoch[2] Train-accuracy=0.843205
INFO:root:Epoch[2] Time cost=0.056
INFO:root:Epoch[2] Validation-accuracy=0.833462
INFO:root:Epoch[3] Train-accuracy=0.850897
INFO:root:Epoch[3] Time cost=0.054
INFO:root:Epoch[3] Validation-accuracy=0.833077
INFO:root:Epoch[4] Train-accuracy=0.860769
INFO:root:Epoch[4] Time cost=0.055
INFO:root:Epoch[4] Validation-accuracy=0.832692
INFO:root:Epoch[5] Train-accuracy=0.860000
INFO:root:Epoch[5] Time cost=0.053
INFO:root:Epoch[5] Validation-accuracy=0.833462
INFO:root:Epoch[6] Train-accuracy=0.863333
INFO:root:Epoch[6] Time cost=0.057
INFO:root:Epoch[6] Validation-accuracy=0.833462
INFO:root:Epoch[7] Train-accuracy=0.862179
INFO:root:Epoch[7] Time cost=0.049
INFO:root:Epoch[7] Validation-accuracy=0

INFO:root:Epoch[64] Train-accuracy=0.843205
INFO:root:Epoch[64] Time cost=0.053
INFO:root:Epoch[64] Validation-accuracy=0.833462
INFO:root:Epoch[65] Train-accuracy=0.843205
INFO:root:Epoch[65] Time cost=0.052
INFO:root:Epoch[65] Validation-accuracy=0.833462
INFO:root:Epoch[66] Train-accuracy=0.843205
INFO:root:Epoch[66] Time cost=0.046
INFO:root:Epoch[66] Validation-accuracy=0.833462
INFO:root:Epoch[67] Train-accuracy=0.843205
INFO:root:Epoch[67] Time cost=0.045
INFO:root:Epoch[67] Validation-accuracy=0.833462
INFO:root:Epoch[68] Train-accuracy=0.843205
INFO:root:Epoch[68] Time cost=0.046
INFO:root:Epoch[68] Validation-accuracy=0.833462
INFO:root:Epoch[69] Train-accuracy=0.843205
INFO:root:Epoch[69] Time cost=0.049
INFO:root:Epoch[69] Validation-accuracy=0.833462
INFO:root:Epoch[70] Train-accuracy=0.843205
INFO:root:Epoch[70] Time cost=0.043
INFO:root:Epoch[70] Validation-accuracy=0.833462
INFO:root:Epoch[71] Train-accuracy=0.843205
INFO:root:Epoch[71] Time cost=0.044
INFO:root:Epoch[7

Accuracy: 83.3461538462 %


In [66]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

mlp = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=mlp, shape=shape)
network_chart=mx.viz.plot_network(symbol=mlp, shape=shape)
network_chart.render('DDNN + SVM')

'DDNN + SVM.pdf'

In [67]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = mlp,         # Use the network we just defined
    label_names = ['svm_label'],
)
mod.fit(
    train_data=train_iter,
    eval_data=test_iter,  
    batch_end_callback = mx.callback.Speedometer(batch_size, 64),  
    num_epoch = 100,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 0.001,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(test_iter, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Train-accuracy=0.829615
INFO:root:Epoch[0] Time cost=0.164
INFO:root:Epoch[0] Validation-accuracy=0.833462
INFO:root:Epoch[1] Train-accuracy=0.843718
INFO:root:Epoch[1] Time cost=0.118
INFO:root:Epoch[1] Validation-accuracy=0.833462
INFO:root:Epoch[2] Train-accuracy=0.843718
INFO:root:Epoch[2] Time cost=0.113
INFO:root:Epoch[2] Validation-accuracy=0.833462
INFO:root:Epoch[3] Train-accuracy=0.843718
INFO:root:Epoch[3] Time cost=0.116
INFO:root:Epoch[3] Validation-accuracy=0.833462
INFO:root:Epoch[4] Train-accuracy=0.843718
INFO:root:Epoch[4] Time cost=0.118
INFO:root:Epoch[4] Validation-accuracy=0.833462
INFO:root:Epoch[5] Train-accuracy=0.843718
INFO:root:Epoch[5] Time cost=0.115
INFO:root:Epoch[5] Validation-accuracy=0.833462
INFO:root:Epoch[6] Train-accuracy=0.843718
INFO:root:Epoch[6] Time cost=0.116
INFO:root:Epoch[6] Validation-accuracy=0.833462
INFO:root:Epoch[7] Train-accuracy=0.843718
INFO:root:Epoch[7] Time cost=0.116
INFO:root:Epoch[7] Validation-accuracy=0

INFO:root:Epoch[64] Train-accuracy=0.843718
INFO:root:Epoch[64] Time cost=0.122
INFO:root:Epoch[64] Validation-accuracy=0.833462
INFO:root:Epoch[65] Train-accuracy=0.843718
INFO:root:Epoch[65] Time cost=0.109
INFO:root:Epoch[65] Validation-accuracy=0.833462
INFO:root:Epoch[66] Train-accuracy=0.843718
INFO:root:Epoch[66] Time cost=0.109
INFO:root:Epoch[66] Validation-accuracy=0.833462
INFO:root:Epoch[67] Train-accuracy=0.843718
INFO:root:Epoch[67] Time cost=0.110
INFO:root:Epoch[67] Validation-accuracy=0.833462
INFO:root:Epoch[68] Train-accuracy=0.843718
INFO:root:Epoch[68] Time cost=0.109
INFO:root:Epoch[68] Validation-accuracy=0.833462
INFO:root:Epoch[69] Train-accuracy=0.843718
INFO:root:Epoch[69] Time cost=0.111
INFO:root:Epoch[69] Validation-accuracy=0.833462
INFO:root:Epoch[70] Train-accuracy=0.843718
INFO:root:Epoch[70] Time cost=0.110
INFO:root:Epoch[70] Validation-accuracy=0.833462
INFO:root:Epoch[71] Train-accuracy=0.843718
INFO:root:Epoch[71] Time cost=0.113
INFO:root:Epoch[7

Accuracy: 83.3461538462 %


In [68]:
# LENET CNN + SVM
data = mx.symbol.Variable('data')
# first conv layer
conv1 = mx.sym.Convolution(data=data, kernel=(5,5), num_filter=32)
tanh1 = mx.sym.Activation(data=conv1, act_type="relu")
pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2,2), stride=(2,2))
# second conv layer
conv2 = mx.sym.Convolution(data=pool1, kernel=(5,5), num_filter=64)
tanh2 = mx.sym.Activation(data=conv2, act_type="relu")
pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2,2), stride=(2,2))
# first fullc layer
flatten = mx.sym.Flatten(data=pool2)
fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=512)
tanh3 = mx.sym.Activation(data=fc1, act_type="relu")
# second fullc
fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=2)
# svm layer
lenet = mx.symbol.SVMOutput(data=fc2, name='svm')

# draw network chart
shape = {"data" : (batch_size, 1, 32, 32)}
mx.viz.plot_network(symbol=lenet, shape=shape)
network_chart=mx.viz.plot_network(symbol=lenet, shape=shape)
network_chart.render('CNN + SVM')

'CNN + SVM.pdf'

In [77]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = lenet,         
    label_names = ['svm_label']
)
mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, 64),  
    num_epoch = 100,       # Train for 100 epochs
        optimizer = 'Adam',

    optimizer_params = {
        'learning_rate': 0.1,  # Learning rate
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Train-accuracy=0.408205
INFO:root:Epoch[0] Time cost=0.342
INFO:root:Epoch[0] Validation-accuracy=0.833462
INFO:root:Epoch[1] Train-accuracy=0.788077
INFO:root:Epoch[1] Time cost=0.324
INFO:root:Epoch[1] Validation-accuracy=0.833462
INFO:root:Epoch[2] Train-accuracy=0.824487
INFO:root:Epoch[2] Time cost=0.322
INFO:root:Epoch[2] Validation-accuracy=0.833462
INFO:root:Epoch[3] Train-accuracy=0.843205
INFO:root:Epoch[3] Time cost=0.328
INFO:root:Epoch[3] Validation-accuracy=0.833462
INFO:root:Epoch[4] Train-accuracy=0.843205
INFO:root:Epoch[4] Time cost=0.328
INFO:root:Epoch[4] Validation-accuracy=0.833462
INFO:root:Epoch[5] Train-accuracy=0.843205
INFO:root:Epoch[5] Time cost=0.348
INFO:root:Epoch[5] Validation-accuracy=0.833462
INFO:root:Epoch[6] Train-accuracy=0.843205
INFO:root:Epoch[6] Time cost=0.371
INFO:root:Epoch[6] Validation-accuracy=0.833462
INFO:root:Epoch[7] Train-accuracy=0.843205
INFO:root:Epoch[7] Time cost=0.325
INFO:root:Epoch[7] Validation-accuracy=0

INFO:root:Epoch[64] Train-accuracy=0.843205
INFO:root:Epoch[64] Time cost=0.322
INFO:root:Epoch[64] Validation-accuracy=0.833462
INFO:root:Epoch[65] Train-accuracy=0.843205
INFO:root:Epoch[65] Time cost=0.320
INFO:root:Epoch[65] Validation-accuracy=0.833462
INFO:root:Epoch[66] Train-accuracy=0.843205
INFO:root:Epoch[66] Time cost=0.320
INFO:root:Epoch[66] Validation-accuracy=0.833462
INFO:root:Epoch[67] Train-accuracy=0.843205
INFO:root:Epoch[67] Time cost=0.321
INFO:root:Epoch[67] Validation-accuracy=0.833462
INFO:root:Epoch[68] Train-accuracy=0.843205
INFO:root:Epoch[68] Time cost=0.321
INFO:root:Epoch[68] Validation-accuracy=0.833462
INFO:root:Epoch[69] Train-accuracy=0.843205
INFO:root:Epoch[69] Time cost=0.329
INFO:root:Epoch[69] Validation-accuracy=0.833462
INFO:root:Epoch[70] Train-accuracy=0.843205
INFO:root:Epoch[70] Time cost=0.362
INFO:root:Epoch[70] Validation-accuracy=0.833462
INFO:root:Epoch[71] Train-accuracy=0.843205
INFO:root:Epoch[71] Time cost=0.344
INFO:root:Epoch[7

Accuracy: 83.3461538462 %
